In [5]:
%load_ext autoreload
%autoreload 2

import analysis
import data
import models
import numpy as np
import os
import pandas as pd
import pingouin as pg
import plotly.express as px
from scipy.stats import ttest_rel
import torch
import warnings

warnings.filterwarnings('ignore')

model_path = 'models'
num_models = 10
num_bootstrap_sims = 10000
num_training_epochs = 30
num_training_epochs_comparison = 10
train_model = False
train_comparison_model = False

num_objects = 350
num_tasks = 36
num_task_context_units = 16
num_context_independent_units = 64
num_context_dependent_units = 128
size_idx = 2541  # Index of the "is_small" feature
size_task_idx = 33  # Index of the size task

isc_models = models.load_isc_models(num_models, train_model, num_training_epochs, model_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
# Extract task context, context independent, and context dependent representations from the models
# Calculate similarity matrices for each representation

task_context_reps = np.zeros((num_models,num_tasks,num_task_context_units))
context_independent_reps = np.zeros((num_models,num_objects,num_context_independent_units))
context_dependent_reps = np.zeros((num_models,num_tasks,num_objects,num_context_dependent_units))
feature_reps = data.load_feature_reps()

task_context_sim = np.zeros((num_models,num_tasks,num_tasks))
context_independent_sim = np.zeros((num_models,num_objects,num_objects))
context_dependent_sim = np.zeros((num_models,num_tasks,num_objects,num_objects))
feature_sim = analysis.calculate_similarity_matrix(feature_reps,method='correlation')
for model_idx,isc_model in enumerate(isc_models):
    task_context_reps[model_idx] = isc_model.get_task_context_reps()
    context_independent_reps[model_idx] = isc_model.get_context_independent_reps()
    context_dependent_reps[model_idx] = isc_model.get_context_dependent_reps()
    task_context_sim[model_idx] = analysis.calculate_similarity_matrix(task_context_reps[model_idx])
    context_independent_sim[model_idx] = analysis.calculate_similarity_matrix(context_independent_reps[model_idx])
    context_dependent_sim[model_idx] = analysis.calculate_similarity_matrix(context_dependent_reps[model_idx])

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


# Part 1: Influence of Control on Semantic Structure

## Experiment 1: Empirical Evaluation of Learned Conceptual Structure

In [9]:
stimuli = pd.read_csv('data/leuven_triplets_meanagree.csv')
stimuli.head()

,Unnamed: 0,tname,head,O1,O2,pChoseO1,pMaj,Winner,ttype,ircosPred,modAcc,twoDPred,twoDAcc,glovePred,gloveAcc
0,v1,bumblebee_ant_spider,bumblebee,ant,spider,0.611111,0.611111,ant,CC,ant,1,ant,1,ant,1.0
1,v2,mosquito_eagle_earwig,mosquito,eagle,earwig,0.297297,0.702703,earwig,CD,earwig,1,earwig,1,eagle,0.0
2,v3,ladybug_earwig_radish,ladybug,earwig,radish,0.567568,0.567568,earwig,COD,earwig,1,earwig,1,earwig,1.0
3,v4,louse_goldfish_squid,louse,goldfish,squid,0.405405,0.594595,squid,DD,goldfish,0,goldfish,0,goldfish,0.0
4,v5,centipede_beaver_t-shirt,centipede,beaver,t-shirt,0.864865,0.864865,beaver,DOD,beaver,1,beaver,1,beaver,1.0


## Analysis 2: Operation of Control on Abstract Semantic Dimensions

In [10]:
contexts_to_keep, objects_to_keep, num_objects_to_keep = analysis.filter_nan_similarity(feature_reps[:-1], feature_sim[:-1]) #exclude naming context

independent_correlations, dependent_correlations = analysis.fast_rsa_bootstrap(
                                                        feature_sim[contexts_to_keep],
                                                        np.repeat(context_independent_sim.mean(axis=0)[np.newaxis,:],len(contexts_to_keep),axis=0),
                                                        context_dependent_sim.mean(axis=0)[contexts_to_keep],
                                                        objects_to_keep,num_objects_to_keep,
                                                        n_sims=num_bootstrap_sims)
print(f'Context independent vs feature representation r={independent_correlations.mean():.2f}')
print(f'Context dependent vs feature representation r={dependent_correlations.mean():.2f}')
print(f'Difference is significant at p={((independent_correlations-dependent_correlations).mean(axis=-1)>0).mean()}')

Context independent vs feature representation r=0.38
Context dependent vs feature representation r=0.52
Difference is significant at p=0.0


## Experiment 2: Empirical Evaluation of Task-specific Conceptual Structure

In [11]:
stimuli = pd.read_csv('data/rt_rotated_2D_embeddings_and_metadata.csv')
stimuli.head()

,Unnamed: 0,Item,Domain,Cat,StandDiam,MnDiam,knd1,knd2,sz1,sz2
0,1,baseball,Artifact,Ball,2.90,2.90,-0.071509,0.103555,0.120037,0.055933
1,2,basketball,Artifact,Ball,9.55,9.55,-0.284986,-0.015083,0.595614,-0.043278
2,3,beach_ball,Artifact,Ball,NaN,16.00,-0.250730,0.005826,0.579408,0.061162
3,4,billiard_ball,Artifact,Ball,2.44,2.44,-0.117420,0.094921,-0.019745,-0.089494
4,5,blueberry,Plant,FV,NaN,0.41,0.310792,0.001737,-0.472101,-0.122361


## Analysis 3: Substructure of Task-Specific Representations

### Correlation of Size with Other Features (by Category)

In [13]:
feature_values = data.get_raw_feature_values().sum(axis=0)
animal_idxs = data.get_item_indices_by_category(['bird','fish','mammal','reptile'])
instrument_idxs = data.get_item_indices_by_category(['instrument'])

animal_fts, instrument_fts = feature_values[animal_idxs], feature_values[instrument_idxs]
animal_sizes, instrument_sizes = feature_values[animal_idxs,size_idx], feature_values[instrument_idxs,size_idx]
ft_mask = (animal_fts.sum(axis=0)>0)|(instrument_fts.sum(axis=0)>0)
animal_fts, instrument_fts = animal_fts[:,ft_mask], instrument_fts[:,ft_mask]

within_corrs, across_corrs = analysis.fast_correlation_splithalf(animal_sizes,animal_fts,
                                                                 instrument_sizes,instrument_fts,
                                                                 num_bootstrap_sims,replace_nan=0)
print(f'Correlation within category r={within_corrs.mean():.2f}')
print(f'Correlation between categories r={across_corrs.mean():.2f}')
print(f'Difference is significant at p={((across_corrs-within_corrs)>0).mean()}')

Correlation within category r=0.56
Correlation between categories r=0.11
Difference is significant at p=0.0


### Angle between Animal and Instrument Size Representations

In [14]:
small_animal_idxs = data.get_item_indices_by_category_and_size(['bird','fish','mammal','reptile'],'small')
large_animal_idxs = data.get_item_indices_by_category_and_size(['bird','fish','mammal','reptile'],'large')
small_instrument_idxs = data.get_item_indices_by_category_and_size(['instrument'],'small')
large_instrument_idxs = data.get_item_indices_by_category_and_size(['instrument'],'large')

within_angles, across_angles = np.zeros((num_models,num_bootstrap_sims)), np.zeros((num_models,num_bootstrap_sims))
for i in range(num_models):
    size_context_reps = isc_models[i].get_context_dependent_reps()[-3]
    small_animal_reps = size_context_reps[small_animal_idxs]
    large_animal_reps = size_context_reps[large_animal_idxs]
    small_instrument_reps = size_context_reps[small_instrument_idxs]
    large_instrument_reps = size_context_reps[large_instrument_idxs]

    within_angles[i], across_angles[i] = analysis.cosine_splithalf(
                                            small_animal_reps, large_animal_reps,
                                            small_instrument_reps, large_instrument_reps,
                                            n_sims=num_bootstrap_sims)

print(f'Angle within category={within_angles.mean():.2f}')
print(f'Angle between categories={across_angles.mean():.2f}')
print(f'Difference is significant at p={((within_angles-across_angles)>0).mean()}')

Angle within category=0.27
Angle between categories=0.46
Difference is significant at p=0.00543


#### Comparison Model

In [15]:
comparison_models = models.load_comparison_models(num_models)

In [16]:
small_animal_idxs = data.get_item_indices_by_category_and_size(['bird','fish','mammal','reptile'],'small')
large_animal_idxs = data.get_item_indices_by_category_and_size(['bird','fish','mammal','reptile'],'large')
small_instrument_idxs = data.get_item_indices_by_category_and_size(['instrument'],'small')
large_instrument_idxs = data.get_item_indices_by_category_and_size(['instrument'],'large')

small_animal_idxs_complement = [i+num_objects for i in small_animal_idxs]
large_animal_idxs_complement = [i+num_objects for i in large_animal_idxs]
small_instrument_idxs_complement = [i+num_objects for i in small_instrument_idxs]
large_instrument_idxs_complement = [i+num_objects for i in large_instrument_idxs]

within_angles, across_angles = np.zeros((num_models,num_bootstrap_sims)), np.zeros((num_models,num_bootstrap_sims))
for i in range(num_models):
    size_context_reps = comparison_models[i].get_context_dependent_reps()[-3]
    small_animal_reps = size_context_reps[small_animal_idxs]
    large_animal_reps = size_context_reps[large_animal_idxs]
    small_instrument_reps = size_context_reps[small_instrument_idxs]
    large_instrument_reps = size_context_reps[large_instrument_idxs]
    small_animal_reps_complement = size_context_reps[small_animal_idxs_complement]
    large_animal_reps_complement = size_context_reps[large_animal_idxs_complement]
    small_instrument_reps_complement = size_context_reps[small_instrument_idxs_complement]
    large_instrument_reps_complement = size_context_reps[large_instrument_idxs_complement]

    within_angles[i], across_angles[i] = analysis.cosine_splithalf_paired(
                                            small_animal_reps, large_animal_reps,
                                            small_instrument_reps, large_instrument_reps,
                                            small_animal_reps_complement,large_animal_reps_complement,
                                            small_instrument_reps_complement,large_instrument_reps_complement,
                                            n_sims=num_bootstrap_sims)

print(f'Angle within category={within_angles.mean():.2f}')
print(f'Angle between categories={across_angles.mean():.2f}')
print(f'Difference is significant at p={((within_angles-across_angles)>0).mean()}')

Angle within category=0.60
Angle between categories=0.66
Difference is significant at p=0.32695


# Part 2: Influence of Semantics on Control

## Analysis 4: Cross-task Similarity of Representations Used for Control

In [17]:
contexts_to_keep, objects_to_keep, num_objects_to_keep = analysis.filter_nan_similarity(feature_reps[:-1], feature_sim[:-1]) #exclude naming context
task_ft_sim = analysis.calculate_task_similarity_matrix(feature_sim,contexts_to_keep,
                                                        objects_to_keep,num_objects_to_keep)

task_context_sim_ = task_context_sim.mean(axis=0)[contexts_to_keep][:,contexts_to_keep][np.triu_indices(len(contexts_to_keep),1)]

correlation, null_correlations = analysis.fast_correlation_vec_vec_bootstrap(task_ft_sim,
                                                                             task_context_sim_,
                                                                             num_bootstrap_sims)
print(f'Correlation between task-context reps and task similarity r={correlation:.3f}')
print(f'Correlation of the null distribution r={null_correlations.mean():.3f}')
print(f'Difference is significant at p={((null_correlations-correlation)>0).mean()}')

Correlation between task-context reps and task similarity r=0.600
Correlation of the null distribution r=0.001
Difference is significant at p=0.0


## Analysis 5: Shaping of Representations Used for Control to Optimize Task Performance

In [18]:
train_x, train_y,experiment_stimulus_indices,small_animal_idxs,large_animal_idxs,small_instrument_idxs,large_instrument_idxs = data.make_full_distractor_training_data()
interleaved_x = (train_x[0].clone(), train_x[1].clone())
interleaved_x[1][:,0] = 0
interleaved_x[1][:,1] = 1

In [19]:
angles_blocked = []
for model_idx in range(10):
    learned_context_model = models.ISCModel(device='mps',num_tasks=2,num_task_context_units=16,num_context_dependent_hidden_units=128)
    learned_context_model.load_old_model_weights(isc_models[0].state_dict(),use_old_size_starting_point = True)
    learned_context_model.metrics.append(models.SizeOrthogonalityMetric())
    for epoch in range(5):
        learned_context_model.train(train_x,train_y,epochs=1)
    angles_blocked.append(learned_context_model.metrics[2].values)

In [20]:
tprs = learned_context_model.metrics[1].values
plot_angles = pd.DataFrame(learned_context_model.metrics[2].values)
plot_angles = plot_angles[plot_angles.color=='across'].y.values
ys = list(tprs)+list(plot_angles)
xs = list(range(len(tprs)))*2
colors = ['True Positive Rate']*len(tprs)+['Cosine Distance']*len(plot_angles)
fig = px.line(x=xs,y=ys,color=colors,)
fig.update_layout(width=1600,height=300,plot_bgcolor='white',
                    legend=dict(
                                font=dict(size=16)
                                )
)
fig.update_xaxes(title='Epoch',showline=True,linewidth=1.5,linecolor='black',mirror=True,ticks='outside',showgrid=True,gridcolor='#dddddd',titlefont=dict(size=20))
fig.update_yaxes(title='Value',showline=True,linewidth=1.5,linecolor='black',mirror=True,ticks='outside',showgrid=True,gridcolor='#dddddd',titlefont=dict(size=20))
fig.show()

In [21]:
angles_interleaved = []
for model_idx in range(10):
    learned_context_model = models.ISCModel(device='mps',num_tasks=2,num_task_context_units=16,num_context_dependent_hidden_units=128)
    learned_context_model.load_old_model_weights(isc_models[0].state_dict(),use_old_size_starting_point = True)
    learned_context_model.metrics.append(models.SizeOrthogonalityMetric())
    for epoch in range(5):
        learned_context_model.train(interleaved_x,train_y,epochs=1)
    angles_interleaved.append(learned_context_model.metrics[2].values)

In [22]:
for name,angles in zip(['blocked','interleaved'],[angles_blocked,angles_interleaved]):
    df = None
    for i in range(10):
        d = pd.DataFrame(angles[i])
        d['model'] = i
        if df is None:
            df = d
        else:
            df = pd.concat([df,d])
    win_start = df[(df.x==0)&(df.color=='within')].y.values
    win_end = df[(df.x==9)&(df.color=='within')].y.values
    acr_start = df[(df.x==0)&(df.color=='across')].y.values
    acr_end = df[(df.x==9)&(df.color=='across')].y.values
    print(f'-----{name}-----')
    print(f'Within from {win_start.mean():.2f} to {win_end.mean():.2f}')
    print(f'Across from {acr_start.mean():.2f} to {acr_end.mean():.2f}')
    print('Ttest within:',ttest_rel(win_start,win_end,alternative='greater'))
    print('Ttest across:',ttest_rel(acr_start,acr_end,alternative='less'))
    print()

-----blocked-----
Within from 0.21 to 0.08
Across from 0.40 to 1.14
Ttest within: TtestResult(statistic=28.312820701346293, pvalue=2.0804393240321406e-10, df=9)
Ttest across: TtestResult(statistic=-14.93860824365336, pvalue=5.84501353155045e-08, df=9)

-----interleaved-----
Within from 0.21 to 0.17
Across from 0.40 to 0.39
Ttest within: TtestResult(statistic=4.101186152245456, pvalue=0.001335982351118196, df=9)
Ttest across: TtestResult(statistic=0.38648866759987144, pvalue=0.6459432231319444, df=9)



## Experiment 3: Effects of Semantics on Representations Used for Control

### Model Training

In [24]:
train_x,train_y,size_conditions,cat_conditions,random_cat_conditions,blocks = data.make_behavioral_experiment_training_data(distractor_strength=.975)

def calc_model_error(model,train_x,train_y,noise=0):
    errors = torch.abs(model(train_x,noise=noise)-train_y)[:,[2541,2542]].mean(axis=-1)
    return errors.cpu().detach().numpy()

error_data = []
for model_idx in range(1):
    simulation_model = models.ISCModel(device='mps',num_tasks=5,num_task_context_units=16,num_context_dependent_hidden_units=128)
    save_file = f'isc_model-{model_idx}-bhvsim.torch'
    if save_file in os.listdir('models'):
        simulation_model.load_state_dict(torch.load(os.path.join('models',save_file)))
    else:
        simulation_model.load_old_model_weights(isc_models[model_idx].state_dict(),use_old_size_starting_point = True)
        errors = calc_model_error(simulation_model,train_x,train_y)
        c=1
        while errors.mean() > 0.18:
            simulation_model.train(train_x,train_y,epochs=1)
            errors = calc_model_error(simulation_model,train_x,train_y)
            c+=1
        print(model_idx, errors.mean(),c)
        torch.save(simulation_model.state_dict(),os.path.join('models',save_file))
    errors = calc_model_error(simulation_model,train_x,train_y)
    accs = ((simulation_model(train_x)[:,2541]>simulation_model(train_x)[:,2542])==train_y[:,2541]).float().cpu().detach().numpy()
    error_data.append(pd.DataFrame({'model':[model_idx]*len(errors),'rt':errors,'error':1-accs,
                                    'size_condition':size_conditions,'cat_condition':cat_conditions,'rand_condition':random_cat_conditions,'block_type':blocks}))
error_data = pd.concat(error_data,axis=0)

0 0.17846923 29


In [27]:
error_data = []
for i in range(71):
    errors = calc_model_error(simulation_model,train_x,train_y,noise=1.2)
    preds = simulation_model(train_x,noise=1.175)
    accs = ((preds[:,2541]>preds[:,2542])==train_y[:,2541]).float().cpu().detach().numpy()
    error_data.append(pd.DataFrame({'model':[i]*len(errors),'rt':errors,'error':1-accs,
                                    'size_condition':size_conditions,'cat_condition':cat_conditions,'rand_condition':random_cat_conditions,'block_type':blocks}))
error_data = pd.concat(error_data,axis=0)
#error_data.to_csv('data/experiment3_simulation_data_0200.csv')

### Model Performance Statistics

In [28]:
model_data = pd.read_csv('data/experiment3_simulation_data_0200.csv') #models that were trained to 0.200 avg error

model_data = model_data[model_data.block_type!='random']
model_data_interleaved = model_data[model_data.block_type=='interleaved']
model_data_blocked = model_data[model_data.block_type=='blocked']

print('Blocked ANOVA')
print('  RT')
display(pg.rm_anova(data=model_data_blocked,dv='rt',
                    within=['size_condition','cat_condition'],subject='model',
                    detailed=True,effsize='n2'))
print('  Error')
display(pg.rm_anova(data=model_data_blocked,dv='error',
                    within=['size_condition','cat_condition'],subject='model',
                    detailed=True,effsize='n2'))
print()
print('Interleaved ANOVA')
print('  RT')
display(pg.rm_anova(data=model_data_interleaved,dv='rt',
                    within=['size_condition','cat_condition'],subject='model',
                    detailed=True,effsize='n2'))
print('  Error')
display(pg.rm_anova(data=model_data_interleaved,dv='error',
                    within=['size_condition','cat_condition'],subject='model',
                    detailed=True,effsize='n2'))

Blocked ANOVA
  RT


,Source,SS,ddof1,ddof2,MS,F,p-unc,p-GG-corr,n2,eps
0,size_condition,0.072149,1,70,0.072149,21.101734,0.000019,0.000019,0.085793,1.0
1,cat_condition,0.009060,1,70,0.009060,4.772670,0.032266,0.032266,0.010773,1.0
2,size_condition * cat_condition,0.017190,1,70,0.017190,7.172434,0.009218,0.009218,0.020441,1.0


  Error


,Source,SS,ddof1,ddof2,MS,F,p-unc,p-GG-corr,n2,eps
0,size_condition,0.987306,1,70,0.987306,372.524886,9.654212e-30,9.654212e-30,0.495588,1.0
1,cat_condition,0.197799,1,70,0.197799,109.105979,6.327899e-16,6.327899e-16,0.099287,1.0
2,size_condition * cat_condition,0.171797,1,70,0.171797,74.760767,1.178961e-12,1.178961e-12,0.086235,1.0



Interleaved ANOVA
  RT


,Source,SS,ddof1,ddof2,MS,F,p-unc,p-GG-corr,n2,eps
0,size_condition,0.199630,1,70,0.199630,68.549625,5.583715e-12,5.583715e-12,0.204404,1.0
1,cat_condition,0.004216,1,70,0.004216,1.577661,2.132740e-01,2.132740e-01,0.004317,1.0
2,size_condition * cat_condition,0.000031,1,70,0.000031,0.010774,9.176284e-01,9.176284e-01,0.000032,1.0


  Error


,Source,SS,ddof1,ddof2,MS,F,p-unc,p-GG-corr,n2,eps
0,size_condition,2.112694,1,70,2.112694,699.433056,3.634149e-38,3.634149e-38,0.762314,1.0
1,cat_condition,0.000047,1,70,0.000047,0.021447,8.839887e-01,8.839887e-01,0.000017,1.0
2,size_condition * cat_condition,0.004536,1,70,0.004536,2.219670,1.407543e-01,1.407543e-01,0.001637,1.0


In [29]:
print('Blocked Simple Effects - RT')
print('    Category Effect @ Size Match')
display(pg.rm_anova(data=model_data_blocked[model_data_blocked.size_condition=='s_ma'],dv='rt',
                    within='cat_condition',subject='model',detailed=True,effsize='n2'))
print('    Category Effect @ Size Mismatch')
display(pg.rm_anova(data=model_data_blocked[model_data_blocked.size_condition=='s_ms'],dv='rt',
                    within='cat_condition',subject='model',detailed=True,effsize='n2'))
print('Blocked Simple Effects - Error')
print('    Category Effect @ Size Match')
display(pg.rm_anova(data=model_data_blocked[model_data_blocked.size_condition=='s_ma'],dv='error',
                    within='cat_condition',subject='model',detailed=True,effsize='n2'))
print('    Category Effect @ Size Mismatch')
display(pg.rm_anova(data=model_data_blocked[model_data_blocked.size_condition=='s_ms'],dv='error',
                    within='cat_condition',subject='model',detailed=True,effsize='n2'))

Blocked Simple Effects - RT
    Category Effect @ Size Match


,Source,SS,DF,MS,F,p-unc,n2,eps
0,cat_condition,0.000645,1,0.000645,0.252791,0.616695,0.001687,1.0
1,Error,0.178723,70,0.002553,NaN,NaN,NaN,NaN


    Category Effect @ Size Mismatch


,Source,SS,DF,MS,F,p-unc,n2,eps
0,cat_condition,0.025604,1,0.025604,14.70048,0.000273,0.066302,1.0
1,Error,0.121921,70,0.001742,NaN,NaN,NaN,NaN


Blocked Simple Effects - Error
    Category Effect @ Size Match


,Source,SS,DF,MS,F,p-unc,n2,eps
0,cat_condition,0.000458,1,0.000458,0.174401,0.677508,0.001307,1.0
1,Error,0.183798,70,0.002626,NaN,NaN,NaN,NaN


    Category Effect @ Size Mismatch


,Source,SS,DF,MS,F,p-unc,n2,eps
0,cat_condition,0.369138,1,0.369138,248.549172,9.937880e-25,0.564091,1.0
1,Error,0.103962,70,0.001485,NaN,NaN,NaN,NaN


In [30]:
print('Blocked vs Interleaved Interference Effect Size')
transformed_data = model_data[model_data.size_condition=='s_ms'] \
                        .groupby(['block_type','model','cat_condition'],as_index=False).mean() \
                        .pivot_table(index='model',columns=['block_type','cat_condition'],values='rt')
blocked_effect_size = transformed_data['blocked','c_ma']-transformed_data['blocked','c_ms']
interleaved_effect_size = transformed_data['interleaved','c_ma']-transformed_data['interleaved','c_ms']
ttest_result = ttest_rel(blocked_effect_size.values,interleaved_effect_size.values)
print(f'RT Means: Blocked={blocked_effect_size.mean():.2f}, Interleaved={interleaved_effect_size.mean():.2f}, t={ttest_result.statistic:.2f}, p={ttest_result.pvalue:.3f}')
transformed_data = model_data[model_data.size_condition=='s_ms'] \
                        .groupby(['block_type','model','cat_condition'],as_index=False).mean() \
                        .pivot_table(index='model',columns=['block_type','cat_condition'],values='error')
blocked_effect_size = transformed_data['blocked','c_ma']-transformed_data['blocked','c_ms']
interleaved_effect_size = transformed_data['interleaved','c_ma']-transformed_data['interleaved','c_ms']
ttest_result = ttest_rel(blocked_effect_size.values,interleaved_effect_size.values)
print(f'Error Means: Blocked={blocked_effect_size.mean():.2f}, Interleaved={interleaved_effect_size.mean():.2f}, t={ttest_result.statistic:.2f}, p={ttest_result.pvalue:.3f}')

Blocked vs Interleaved Interference Effect Size
RT Means: Blocked=0.03, Interleaved=-0.01, t=3.04, p=0.003
Error Means: Blocked=0.10, Interleaved=0.01, t=8.78, p=0.000


### Behavioral Performance Statistics

In [32]:
behavioral_data = data.load_behavioral_data()
behavioral_data['rt'] = np.log(behavioral_data.rt.values)

acc_data = behavioral_data[behavioral_data.participant_type!='random']
acc_data = acc_data.groupby(['block_type_agg','size_condition','cat_condition','participant'],as_index=False).mean()
acc_data_interleaved = acc_data[acc_data.block_type_agg=='interleaved']
acc_data_blocked = acc_data[acc_data.block_type_agg=='blocked']

rt_data = behavioral_data[(behavioral_data.participant_type!='random')&(behavioral_data.correct==1)]
rt_data = rt_data.groupby(['block_type_agg','size_condition','cat_condition','participant'],as_index=False).mean()
rt_data_interleaved = rt_data[rt_data.block_type_agg=='interleaved']
rt_data_blocked = rt_data[rt_data.block_type_agg=='blocked']

print('Interleaved ANOVA')
print('  RT')
display(pg.rm_anova(data=rt_data_interleaved,dv='rt',
                    within=['size_condition','cat_condition'],subject='participant',
                    detailed=True,effsize='n2'))
print('  Acc')
display(pg.rm_anova(data=acc_data_interleaved,dv='correct',
                    within=['size_condition','cat_condition'],subject='participant',
                    detailed=True,effsize='n2'))
print()
print('Blocked ANOVA')
print('  RT')
display(pg.rm_anova(data=rt_data_blocked,dv='rt',
                    within=['size_condition','cat_condition'],subject='participant',
                    detailed=True,effsize='n2'))
print('  Acc')
display(pg.rm_anova(data=acc_data_blocked,dv='correct',
                    within=['size_condition','cat_condition'],subject='participant',
                    detailed=True,effsize='n2'))
print()

Interleaved ANOVA
  RT


,Source,SS,ddof1,ddof2,MS,F,p-unc,p-GG-corr,n2,eps
0,size_condition,0.064678,1,70,0.064678,57.505461,1.068096e-10,1.068096e-10,0.020802,1.0
1,cat_condition,0.004718,1,70,0.004718,4.039500,4.830266e-02,4.830266e-02,0.001517,1.0
2,size_condition * cat_condition,0.004111,1,70,0.004111,3.770766,5.618090e-02,5.618090e-02,0.001322,1.0


  Acc


,Source,SS,ddof1,ddof2,MS,F,p-unc,p-GG-corr,n2,eps
0,size_condition,0.117307,1,70,0.117307,51.706233,5.602530e-10,5.602530e-10,0.096762,1.0
1,cat_condition,0.000084,1,70,0.000084,0.077462,7.815874e-01,7.815874e-01,0.000069,1.0
2,size_condition * cat_condition,0.000755,1,70,0.000755,0.934147,3.371152e-01,3.371152e-01,0.000623,1.0



Blocked ANOVA
  RT


,Source,SS,ddof1,ddof2,MS,F,p-unc,p-GG-corr,n2,eps
0,size_condition,0.061523,1,70,0.061523,62.771794,2.533654e-11,2.533654e-11,0.014595,1.0
1,cat_condition,0.009789,1,70,0.009789,8.708691,4.305551e-03,4.305551e-03,0.002322,1.0
2,size_condition * cat_condition,0.012717,1,70,0.012717,13.140782,5.440365e-04,5.440365e-04,0.003017,1.0


  Acc


,Source,SS,ddof1,ddof2,MS,F,p-unc,p-GG-corr,n2,eps
0,size_condition,0.063740,1,70,0.063740,36.362868,6.930197e-08,6.930197e-08,0.054180,1.0
1,cat_condition,0.004860,1,70,0.004860,5.529056,2.152324e-02,2.152324e-02,0.004131,1.0
2,size_condition * cat_condition,0.012439,1,70,0.012439,15.982581,1.562985e-04,1.562985e-04,0.010573,1.0


In [33]:
print('Blocked Simple Effects')
print('  RT')
print('    Category Effect @ Size Match')
display(pg.rm_anova(data=rt_data_blocked[rt_data_blocked.size_condition=='s_ma'],dv='rt',
                    within='cat_condition',subject='participant',detailed=True,effsize='n2'))
print('    Category Effect @ Size Mismatch')
display(pg.rm_anova(data=rt_data_blocked[rt_data_blocked.size_condition=='s_ms'],dv='rt',
                    within='cat_condition',subject='participant',detailed=True,effsize='n2'))
print('  Acc')
print('    Category Effect @ Size Match')
display(pg.rm_anova(data=acc_data_blocked[acc_data_blocked.size_condition=='s_ma'],dv='correct',
                    within='cat_condition',subject='participant',detailed=True,effsize='n2'))
print('    Category Effect @ Size Mismatch')
display(pg.rm_anova(data=acc_data_blocked[acc_data_blocked.size_condition=='s_ms'],dv='correct',
                    within='cat_condition',subject='participant',detailed=True,effsize='n2'))

Blocked Simple Effects
  RT
    Category Effect @ Size Match


,Source,SS,DF,MS,F,p-unc,n2,eps
0,cat_condition,0.000096,1,0.000096,0.082968,0.774166,0.000046,1.0
1,Error,0.080707,70,0.001153,NaN,NaN,NaN,NaN


    Category Effect @ Size Mismatch


,Source,SS,DF,MS,F,p-unc,n2,eps
0,cat_condition,0.022411,1,0.022411,23.869877,0.000006,0.010713,1.0
1,Error,0.065721,70,0.000939,NaN,NaN,NaN,NaN


  Acc
    Category Effect @ Size Match


,Source,SS,DF,MS,F,p-unc,n2,eps
0,cat_condition,0.000874,1,0.000874,1.229002,0.271396,0.001571,1.0
1,Error,0.049800,70,0.000711,NaN,NaN,NaN,NaN


    Category Effect @ Size Mismatch


,Source,SS,DF,MS,F,p-unc,n2,eps
0,cat_condition,0.016424,1,0.016424,17.365769,0.000087,0.02953,1.0
1,Error,0.066204,70,0.000946,NaN,NaN,NaN,NaN


In [34]:
from scipy.stats import ttest_rel
print('Blocked vs Interleaved Interference Effect Size')
transformed_data = rt_data[rt_data.size_condition=='s_ms'] \
                        .groupby(['block_type_agg','participant','cat_condition'],as_index=False).mean() \
                        .pivot_table(index='participant',columns=['block_type_agg','cat_condition'],values='rt')
blocked_effect_size = transformed_data['blocked','c_ma']-transformed_data['blocked','c_ms']
interleaved_effect_size = transformed_data['interleaved','c_ma']-transformed_data['interleaved','c_ms']
ttest_result = ttest_rel(blocked_effect_size.values,interleaved_effect_size.values)
print(f'RT - means: Blocked={blocked_effect_size.mean():.2f}, Interleaved={interleaved_effect_size.mean():.2f}, t={ttest_result.statistic:.2f}, p={ttest_result.pvalue:.3f}')

transformed_data = acc_data[acc_data.size_condition=='s_ms'] \
                        .groupby(['block_type_agg','participant','cat_condition'],as_index=False).mean() \
                        .pivot_table(index='participant',columns=['block_type_agg','cat_condition'],values='correct')
blocked_effect_size = transformed_data['blocked','c_ma']-transformed_data['blocked','c_ms']
interleaved_effect_size = transformed_data['interleaved','c_ma']-transformed_data['interleaved','c_ms']
ttest_result = ttest_rel(blocked_effect_size.values,interleaved_effect_size.values)
print(f'Acc - means: Blocked={blocked_effect_size.mean():.2f}, Interleaved={interleaved_effect_size.mean():.2f}, t={ttest_result.statistic:.2f}, p={ttest_result.pvalue:.3f}')

Blocked vs Interleaved Interference Effect Size
RT - means: Blocked=0.03, Interleaved=-0.00, t=3.25, p=0.002
Acc - means: Blocked=-0.02, Interleaved=-0.00, t=-2.71, p=0.008


### Plots

In [35]:
behavioral_data = data.load_behavioral_data()
behavioral_data['rt'] = np.log(behavioral_data.rt.values)

acc_data = behavioral_data[behavioral_data.participant_type!='random']
acc_data = acc_data.groupby(['block_type_agg','size_condition','cat_condition','participant'],as_index=False).mean()
acc_data_interleaved = acc_data[acc_data.block_type_agg=='interleaved']
acc_data_blocked = acc_data[acc_data.block_type_agg=='blocked']

rt_data = behavioral_data[(behavioral_data.participant_type!='random')&(behavioral_data.correct==1)]
rt_data = rt_data.groupby(['block_type_agg','size_condition','cat_condition','participant'],as_index=False).mean()
rt_data_interleaved = rt_data[rt_data.block_type_agg=='interleaved']
rt_data_blocked = rt_data[rt_data.block_type_agg=='blocked']

plot_data_interleaved = data.add_within_subject_error_bars(rt_data_interleaved)
plot_data_interleaved = plot_data_interleaved.groupby(['cat_condition','size_condition'],as_index=False).mean()
plot_data_interleaved['cat_condition'] = plot_data_interleaved.cat_condition.replace({'c_ma':'Category Match','c_ms':'Category Mismatch'})
plot_data_interleaved['size_condition'] = plot_data_interleaved.size_condition.replace({'s_ma':'Size Match','s_ms':'Size Mismatch'})
plot_data_blocked = data.add_within_subject_error_bars(rt_data_blocked)
plot_data_blocked = plot_data_blocked.groupby(['cat_condition','size_condition'],as_index=False).mean()
plot_data_blocked['cat_condition'] = plot_data_blocked.cat_condition.replace({'c_ma':'Category Match','c_ms':'Category Mismatch'})
plot_data_blocked['size_condition'] = plot_data_blocked.size_condition.replace({'s_ma':'Size Match','s_ms':'Size Mismatch'})

model_data_interleaved = model_data_interleaved.groupby(['model','cat_condition','size_condition'],as_index=False).mean()
model_data_blocked = model_data_blocked.groupby(['model','cat_condition','size_condition'],as_index=False).mean()
plot_data_interleaved_model = data.add_within_subject_error_bars(model_data_interleaved,subject='model',dv='rt',remove_mean=True)
plot_data_interleaved_model = plot_data_interleaved_model.groupby(['cat_condition','size_condition'],as_index=False).mean()
plot_data_interleaved_model['cat_condition'] = plot_data_interleaved_model.cat_condition.replace({'c_ma':'Category Match','c_ms':'Category Mismatch'})
plot_data_interleaved_model['size_condition'] = plot_data_interleaved_model.size_condition.replace({'s_ma':'Size Match','s_ms':'Size Mismatch'})
plot_data_blocked_model = data.add_within_subject_error_bars(model_data_blocked,subject='model',dv='rt',remove_mean=True)
plot_data_blocked_model = plot_data_blocked_model.groupby(['cat_condition','size_condition'],as_index=False).mean()
plot_data_blocked_model['cat_condition'] = plot_data_blocked_model.cat_condition.replace({'c_ma':'Category Match','c_ms':'Category Mismatch'})
plot_data_blocked_model['size_condition'] = plot_data_blocked_model.size_condition.replace({'s_ma':'Size Match','s_ms':'Size Mismatch'})
plot_data_interleaved_acc_model = data.add_within_subject_error_bars(model_data_interleaved,subject='model',dv='error',remove_mean=True)
plot_data_interleaved_acc_model = plot_data_interleaved_acc_model.groupby(['cat_condition','size_condition'],as_index=False).mean()
plot_data_interleaved_acc_model['cat_condition'] = plot_data_interleaved_acc_model.cat_condition.replace({'c_ma':'Category Match','c_ms':'Category Mismatch'})
plot_data_interleaved_acc_model['size_condition'] = plot_data_interleaved_acc_model.size_condition.replace({'s_ma':'Size Match','s_ms':'Size Mismatch'})
plot_data_blocked_acc_model = data.add_within_subject_error_bars(model_data_blocked,subject='model',dv='error',remove_mean=True)
plot_data_blocked_acc_model = plot_data_blocked_acc_model.groupby(['cat_condition','size_condition'],as_index=False).mean()
plot_data_blocked_acc_model['cat_condition'] = plot_data_blocked_acc_model.cat_condition.replace({'c_ma':'Category Match','c_ms':'Category Mismatch'})
plot_data_blocked_acc_model['size_condition'] = plot_data_blocked_acc_model.size_condition.replace({'s_ma':'Size Match','s_ms':'Size Mismatch'})



acc_data_interleaved['error'] = 1-acc_data_interleaved.correct
plot_data_interleaved_acc = data.add_within_subject_error_bars(acc_data_interleaved,dv='error',remove_mean=True)
plot_data_interleaved_acc = plot_data_interleaved_acc.groupby(['cat_condition','size_condition'],as_index=False).mean()
plot_data_interleaved_acc['cat_condition'] = plot_data_interleaved_acc.cat_condition.replace({'c_ma':'Category Match','c_ms':'Category Mismatch'})
plot_data_interleaved_acc['size_condition'] = plot_data_interleaved_acc.size_condition.replace({'s_ma':'Size Match','s_ms':'Size Mismatch'})

acc_data_blocked['error'] = 1-acc_data_blocked.correct
plot_data_blocked_acc = data.add_within_subject_error_bars(acc_data_blocked,dv='error',remove_mean=True)
plot_data_blocked_acc = plot_data_blocked_acc.groupby(['cat_condition','size_condition'],as_index=False).mean()
plot_data_blocked_acc['cat_condition'] = plot_data_blocked_acc.cat_condition.replace({'c_ma':'Category Match','c_ms':'Category Mismatch'})
plot_data_blocked_acc['size_condition'] = plot_data_blocked_acc.size_condition.replace({'s_ma':'Size Match','s_ms':'Size Mismatch'})

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px


model_rt_interleaved = px.bar(plot_data_interleaved_model,x='size_condition',color='cat_condition',
       y='rt_normalized',barmode='group',error_y='rt_error')
human_rt_interleaved = px.bar(plot_data_interleaved,x='size_condition',color='cat_condition',
       y='rt_normalized',barmode='group',error_y='rt_error')
model_rt_blocked = px.bar(plot_data_blocked_model,x='size_condition',color='cat_condition',
       y='rt_normalized',barmode='group',error_y='rt_error')
human_rt_blocked = px.bar(plot_data_blocked,x='size_condition',color='cat_condition',
       y='rt_normalized',barmode='group',error_y='rt_error')
human_acc_interleaved = px.bar(plot_data_interleaved_acc,x='size_condition',color='cat_condition',
       y='error_normalized',barmode='group',error_y='error_error')
human_acc_blocked = px.bar(plot_data_blocked_acc,x='size_condition',color='cat_condition',
       y='error_normalized',barmode='group',error_y='error_error')
model_acc_interleaved = px.bar(plot_data_interleaved_acc_model,x='size_condition',color='cat_condition',
       y='error_normalized',barmode='group',error_y='error_error')
model_acc_blocked = px.bar(plot_data_blocked_acc_model,x='size_condition',color='cat_condition',
       y='error_normalized',barmode='group',error_y='error_error')

fig = make_subplots(rows=2,cols=2,row_titles=['Categorically Blocked Condition','Interleaved Condition'],
                    column_titles=['Model Performance','Human Performance'])
fig.add_trace(model_rt_interleaved.data[0],row=2,col=1)
fig.add_trace(model_rt_interleaved.data[1],row=2,col=1)
fig.add_trace(human_rt_interleaved.data[0],row=2,col=2)
fig.add_trace(human_rt_interleaved.data[1],row=2,col=2)
fig.add_trace(model_rt_blocked.data[0],row=1,col=1)
fig.add_trace(model_rt_blocked.data[1],row=1,col=1)
fig.add_trace(human_rt_blocked.data[0],row=1,col=2)
fig.add_trace(human_rt_blocked.data[1],row=1,col=2)
for idx,trace in enumerate(fig.data):
    if idx > 1:
        trace.showlegend = False
fig.update_layout(title='Log Reaction Time (log ms, Relative to Mean) by Condition',yaxis_side='left',width=1200,height=800,
                  titlefont=dict(size=20))
fig.update_layout(plot_bgcolor='white',title_x=0.5,
                     legend=dict(yanchor='top',y=.99,xanchor='left',x=.01,font=dict(size=16)))
fig.update_xaxes(showline=True,linewidth=1.5,linecolor='black',tickfont=dict(size=16),
              mirror=True,ticks='outside',showgrid=False,titlefont=dict(size=20)
              )
fig.update_yaxes(showline=True,linewidth=1.5,linecolor='black',
              mirror=True,ticks='outside',showgrid=False,titlefont=dict(size=20),
              zeroline=True,zerolinecolor='black',zerolinewidth=1)
#fig.add_annotation(dict(x=0,y=.25,xref='x1',yref='y1',text='test',showarrow=False))
fig['layout']['annotations'][2].update(text="Categorically Blocked Condition",x=-.07,textangle=-90)
fig['layout']['annotations'][3].update(text="Interleaved Condition",x=-.07,textangle=-90)
fig.show()

fig = make_subplots(rows=2,cols=2,row_titles=['Categorically Blocked Condition','Interleaved Condition'],
                    column_titles=['Model Performance','Human Performance'])
fig.add_trace(human_acc_interleaved.data[0],row=2,col=2)
fig.add_trace(human_acc_interleaved.data[1],row=2,col=2)
fig.add_trace(model_acc_interleaved.data[0],row=2,col=1)
fig.add_trace(model_acc_interleaved.data[1],row=2,col=1)
fig.add_trace(human_acc_blocked.data[0],row=1,col=2)
fig.add_trace(human_acc_blocked.data[1],row=1,col=2)
fig.add_trace(model_acc_blocked.data[0],row=1,col=1)
fig.add_trace(model_acc_blocked.data[1],row=1,col=1)
for idx,trace in enumerate(fig.data):
    if idx > 1:
        trace.showlegend = False
fig.update_layout(title='Error Rate (%, Relative to Mean) by Condition',yaxis_side='left',width=1200,height=800,
                  titlefont=dict(size=20))
fig.update_layout(plot_bgcolor='white',title_x=0.5,
                     legend=dict(yanchor='top',y=.99,xanchor='left',x=.01,font=dict(size=16)))
fig.update_xaxes(showline=True,linewidth=1.5,linecolor='black',tickfont=dict(size=16),
              mirror=True,ticks='outside',showgrid=False,titlefont=dict(size=20)
              )
fig.update_yaxes(showline=True,linewidth=1.5,linecolor='black',
              mirror=True,ticks='outside',showgrid=False,titlefont=dict(size=20),
              zeroline=True,zerolinecolor='black',zerolinewidth=1)
#fig.add_annotation(dict(x=0,y=.25,xref='x1',yref='y1',text='test',showarrow=False))
fig['layout']['annotations'][2].update(text="Categorically Blocked Condition",x=-.07,textangle=-90)
fig['layout']['annotations'][3].update(text="Interleaved Condition",x=-.07,textangle=-90)
fig.show()

# Supplementary Information

## Experiment 3 Control: Response-Set Blocking

### Behavioral Performance Statistics

In [39]:
behavioral_data = data.load_behavioral_data()
x = behavioral_data[behavioral_data.participant_type=='random']
set(x[x.block_type=='random1'].target.values)

{'cello',
 'elephant',
 'goldfish',
 'harp',
 'iguana',
 'mouse',
 'piano',
 'recorder',
 'shark',
 'triangle'}

In [40]:
behavioral_data = data.load_behavioral_data()
behavioral_data['rt'] = np.log(behavioral_data.rt)

acc_data = behavioral_data[behavioral_data.participant_type=='random']
acc_data = acc_data.groupby(['block_type_agg','size_condition','rand_condition','participant'],as_index=False).mean()
acc_data_interleaved = acc_data[acc_data.block_type_agg=='interleaved']
acc_data_blocked = acc_data[acc_data.block_type_agg=='random']

rt_data = behavioral_data[(behavioral_data.participant_type=='random')&(behavioral_data.correct==1)]
rt_data = rt_data.groupby(['block_type_agg','size_condition','rand_condition','participant'],as_index=False).mean()
rt_data_interleaved = rt_data[rt_data.block_type_agg=='interleaved']
rt_data_blocked = rt_data[rt_data.block_type_agg=='random']


print('Response-Set Blocked ANOVA')
print('  RT')
display(pg.rm_anova(data=rt_data_blocked,dv='rt',
                    within=['size_condition','rand_condition'],subject='participant',
                    detailed=True,effsize='n2'))
print('  Acc')
display(pg.rm_anova(data=acc_data_blocked,dv='correct',
                    within=['size_condition','rand_condition'],subject='participant',
                    detailed=True,effsize='n2'))
print()

Response-Set Blocked ANOVA
  RT


,Source,SS,ddof1,ddof2,MS,F,p-unc,p-GG-corr,n2,eps
0,size_condition,0.018216,1,15,0.018216,41.631578,0.000011,0.000011,0.021137,1.0
1,rand_condition,0.000001,1,15,0.000001,0.004713,0.946175,0.946175,0.000001,1.0
2,size_condition * rand_condition,0.001126,1,15,0.001126,1.614600,0.223201,0.223201,0.001307,1.0


  Acc


,Source,SS,ddof1,ddof2,MS,F,p-unc,p-GG-corr,n2,eps
0,size_condition,0.012987,1,15,0.012987,28.725378,0.000080,0.000080,0.067785,1.0
1,rand_condition,0.000986,1,15,0.000986,2.718474,0.119973,0.119973,0.005145,1.0
2,size_condition * rand_condition,0.000326,1,15,0.000326,0.668583,0.426344,0.426344,0.001702,1.0


### Plots

In [41]:
behavioral_data['cat_condition'] = behavioral_data['rand_condition']

acc_data = behavioral_data[behavioral_data.participant_type=='random']
acc_data = acc_data.groupby(['block_type_agg','size_condition','cat_condition','participant'],as_index=False).mean()
acc_data_interleaved = acc_data[acc_data.block_type_agg=='interleaved']
acc_data_blocked = acc_data[acc_data.block_type_agg=='random']

rt_data = behavioral_data[(behavioral_data.participant_type=='random')&(behavioral_data.correct==1)]
rt_data = rt_data.groupby(['block_type_agg','size_condition','cat_condition','participant'],as_index=False).mean()
rt_data_interleaved = rt_data[rt_data.block_type_agg=='interleaved']
rt_data_blocked = rt_data[rt_data.block_type_agg=='random']

plot_data_interleaved = data.add_within_subject_error_bars(rt_data_interleaved)
plot_data_interleaved = plot_data_interleaved.groupby(['cat_condition','size_condition'],as_index=False).mean()
plot_data_interleaved['cat_condition'] = plot_data_interleaved.cat_condition.replace({'a_ma':'Response-Set Match','a_ms':'Response-Set Mismatch'})
plot_data_interleaved['size_condition'] = plot_data_interleaved.size_condition.replace({'s_ma':'Size Match','s_ms':'Size Mismatch'})
plot_data_blocked = data.add_within_subject_error_bars(rt_data_blocked)
plot_data_blocked = plot_data_blocked.groupby(['cat_condition','size_condition'],as_index=False).mean()
plot_data_blocked['cat_condition'] = plot_data_blocked.cat_condition.replace({'a_ma':'Response-Set Match','a_ms':'Response-Set Mismatch'})
plot_data_blocked['size_condition'] = plot_data_blocked.size_condition.replace({'s_ma':'Size Match','s_ms':'Size Mismatch'})


acc_data_interleaved['error'] = 1-acc_data_interleaved.correct
plot_data_interleaved_acc = data.add_within_subject_error_bars(acc_data_interleaved,dv='error',remove_mean=True)
plot_data_interleaved_acc = plot_data_interleaved_acc.groupby(['cat_condition','size_condition'],as_index=False).mean()
plot_data_interleaved_acc['cat_condition'] = plot_data_interleaved_acc.cat_condition.replace({'a_ma':'Response-Set Match','a_ms':'Response-Set Mismatch'})
plot_data_interleaved_acc['size_condition'] = plot_data_interleaved_acc.size_condition.replace({'s_ma':'Size Match','s_ms':'Size Mismatch'})

acc_data_blocked['error'] = 1-acc_data_blocked.correct
plot_data_blocked_acc = data.add_within_subject_error_bars(acc_data_blocked,dv='error',remove_mean=True)
plot_data_blocked_acc = plot_data_blocked_acc.groupby(['cat_condition','size_condition'],as_index=False).mean()
plot_data_blocked_acc['cat_condition'] = plot_data_blocked_acc.cat_condition.replace({'a_ma':'Response-Set Match','a_ms':'Response-Set Mismatch'})
plot_data_blocked_acc['size_condition'] = plot_data_blocked_acc.size_condition.replace({'s_ma':'Size Match','s_ms':'Size Mismatch'})

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px


human_rt_interleaved = px.bar(plot_data_interleaved,x='size_condition',color='cat_condition',
       y='rt_normalized',barmode='group',error_y='rt_error')
human_rt_blocked = px.bar(plot_data_blocked,x='size_condition',color='cat_condition',
       y='rt_normalized',barmode='group',error_y='rt_error')
human_acc_interleaved = px.bar(plot_data_interleaved_acc,x='size_condition',color='cat_condition',
       y='error_normalized',barmode='group',error_y='error_error')
human_acc_blocked = px.bar(plot_data_blocked_acc,x='size_condition',color='cat_condition',
       y='error_normalized',barmode='group',error_y='error_error')

fig = make_subplots(rows=1,cols=2,column_titles=['Response-Set Blocked Condition','Interleaved Condition'])
fig.add_trace(human_rt_interleaved.data[0],row=1,col=1)
fig.add_trace(human_rt_interleaved.data[1],row=1,col=1)
fig.add_trace(human_rt_blocked.data[0],row=1,col=2)
fig.add_trace(human_rt_blocked.data[1],row=1,col=2)
for idx,trace in enumerate(fig.data):
    if idx > 1:
        trace.showlegend = False
fig.update_layout(title='Log Reaction Time (log ms, Relative to Mean) by Condition',yaxis_side='left',width=1200,height=400,
                  titlefont=dict(size=20))
fig.update_layout(plot_bgcolor='white',title_x=0.5,
                     legend=dict(yanchor='top',y=.99,xanchor='left',x=.01,font=dict(size=16)))
fig.update_xaxes(showline=True,linewidth=1.5,linecolor='black',tickfont=dict(size=16),
              mirror=True,ticks='outside',showgrid=False,titlefont=dict(size=20)
              )
fig.update_yaxes(showline=True,linewidth=1.5,linecolor='black',
              mirror=True,ticks='outside',showgrid=False,titlefont=dict(size=20),
              zeroline=True,zerolinecolor='black',zerolinewidth=1)
fig.show()

fig = make_subplots(rows=1,cols=2,column_titles=['Response-Set Blocked Condition','Interleaved Condition'])
fig.add_trace(human_acc_interleaved.data[0],row=1,col=1)
fig.add_trace(human_acc_interleaved.data[1],row=1,col=1)
fig.add_trace(human_acc_blocked.data[0],row=1,col=2)
fig.add_trace(human_acc_blocked.data[1],row=1,col=2)
for idx,trace in enumerate(fig.data):
    if idx > 1:
        trace.showlegend = False
fig.update_layout(title='Error Rate (%, Relative to Mean) by Condition',yaxis_side='left',width=1200,height=400,
                  titlefont=dict(size=20))
fig.update_layout(plot_bgcolor='white',title_x=0.5,
                     legend=dict(yanchor='top',y=.99,xanchor='left',x=.01,font=dict(size=16)))
fig.update_xaxes(showline=True,linewidth=1.5,linecolor='black',tickfont=dict(size=16),
              mirror=True,ticks='outside',showgrid=False,titlefont=dict(size=20)
              )
fig.update_yaxes(showline=True,linewidth=1.5,linecolor='black',
              mirror=True,ticks='outside',showgrid=False,titlefont=dict(size=20),
              zeroline=True,zerolinecolor='black',zerolinewidth=1)
fig.show()